In [1]:
import numpy as np
import cv2

In [2]:
#Video capture
cap = cv2.VideoCapture(r"C:\Users\Rahul Dusad\ML Projects\Object Tracking\video\test_video.mp4")

In [3]:
#Read and capture and get first frame
ret, frame = cap.read()

In [4]:
#Set up the initital tracking window
face_casc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_rects = face_casc.detectMultiScale(frame)

In [5]:
#Convert the first list of tuple
face_x, face_y, w, h = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)

In [6]:
#Set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]

In [7]:
#hsv color mapping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

In [8]:
#histogram to target on each frame for the meanshift calculation
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])

In [9]:
#Normalize the histogram
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX);

In [10]:
#Set the termination criteria (10 iterations or move point)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [11]:
while True:
    #Read and capture the first frame
    ret, frame = cap.read()
    
    if ret==True:
        #Convert rgb to hsv
        hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        
        #Calculate the base of roi
        dest_roi = cv2.calcBlackProject([hsv], [0], roi_hist, [0,180], 1)
        
        #Meanshift to get the new coordinates of the rectangle
        ret, track_window = cv2.meanShift(dest_roi, track_window, term_crit)
        
        #Draw new rectangle on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, (0,255,0), 5)
        
        #Open new window and display
        cv2.imshow('Cam Shift', img2)
        
        #Close the window
        if cv2.waitKey(50) & 0xFF == 27:
            break
            
    else:
        break

        


AttributeError: module 'cv2.cv2' has no attribute 'calcBlackProject'

In [ ]:
#Release and destroy
cap.release()
cv2.destroyAllWindows()